## Script to generate augmented images from given images

## Random augmentations with the following: 
- Horizontal and vertical flips
- Crops
- Linear Contrast changing
- Gaussian Noise addition
- Gaussian Blur
- Pixel multiplication (colour change)
- Affine transformations: scale, translate, rotate, shear)

In [1]:
import os
import shutil
import glob
import imgaug as ia
from imgaug import augmenters as iaa
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from keras.models import Model, load_model
from keras.layers.merge import concatenate, add
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from PIL import Image

Using TensorFlow backend.


In [2]:
#this method takes in the path of the directory containing images and their respective masks, and returns a 4d array with their respective values
def get_images_to_augment(path, im_height=256, im_width=256):
    # get directory path for unaugmented images
    imgs_path = path + "images_unaugmented"
    print("Retrieving images from " + imgs_path)

    # get directory path for unaugmented masks
    mask_path = path + "masks_unaugmented"
    print("Retrieving masks from " + mask_path)
    
    imgs = os.listdir(imgs_path)
    masks = os.listdir(mask_path)

    # remove .DS_Store, if it exists in either directory
    if ".DS_Store" in imgs:
        imgs.remove('.DS_Store')

    if ".DS_Store" in masks:
        masks.remove('.DS_Store')

    # remove directory .ipynb_checkpoints if it exists
    if ".ipynb_checkpoints" in imgs:
        shutil.rmtree(imgs_path + '/.ipynb_checkpoints')

    if ".ipynb_checkpoints" in masks:
        shutil.rmtree(mask_path + '/.ipynb_checkpoints')

    # arrays are 4d numpy array of shape (N, height, width, channels)
    im_arr = np.zeros((len(imgs), im_height, im_width, 3), dtype=np.uint8)

    msk_arr = np.zeros((len(masks), im_height, im_width, 1), dtype=np.uint8)
    print('Getting and resizing images ... ')
    for n, id_ in tqdm_notebook(enumerate(imgs), total=len(imgs)):
        #Load images
        img = load_img(imgs_path + '/' + id_)
        image = img_to_array(img)
        image = resize(image, (256, 256, 3), mode='constant', preserve_range=True)

        #Load masks
        msk = load_img(mask_path + '/' + id_, color_mode="grayscale")
        mask = img_to_array(msk)
        mask = resize(mask, (256, 256, 1), mode='constant', preserve_range=True)


        im_arr[n] = image
        msk_arr[n] = mask
    # X now contains the respective values of the pixel intensities in the image
    print('Done!')
  
    return im_arr, msk_arr

    

In [3]:
#get data 
#path = './data/test/'
path = './data/duct_test/new_dataset/'
img_arr, masks_arr = get_images_to_augment(path)
print(img_arr.shape)
print(masks_arr.shape)


Retrieving images from ./data/duct_test/new_dataset/images_unaugmented
Retrieving masks from ./data/duct_test/new_dataset/masks_unaugmented
Getting and resizing images ... 

Done!
(158, 256, 256, 3)
(158, 256, 256, 1)


## Light augmentation

In [4]:
#light augmentation
ia.seed(1)
#sequence of random augmentations
# Image-specific sequence
seq_light_img = iaa.Sequential([
    iaa.Fliplr(0.5, random_state=1), # horizontal flips
    iaa.Flipud(0.5, random_state=2), # vertical flips
    iaa.Crop(percent=(0, 0.1), random_state=3), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    iaa.Sometimes(0.5,
          iaa.GaussianBlur(sigma=(0, 0.5))),

    # Strengthen or weaken the contrast in each image.
    iaa.contrast.LinearContrast((0.5, 1.25), random_state=4),
    # Add gaussian noise for 50% of the images.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    iaa.Sometimes(0.5, 
        iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), random_state=5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2, random_state=6),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # random scaling from 80% to 120% of original size
        translate_percent={"x": (0.0, 0.2), "y": (0.0, 0.2)},# translation from 0 translation to 20% of axis size
        rotate=(-360, 360), # rotate the image randomly between -360 and 360 degrees
        shear=(-45, 45),
        cval=255,#set cval to 255 to prevent any black areas occuring 
        mode='constant', random_state=7
    )
], random_state=8) # apply augmenters in random order





In [5]:
# Mask-specific sequence
seq_light_mask = iaa.Sequential([
    iaa.Fliplr(0.5, random_state=1), # horizontal flips
    iaa.Flipud(0.5, random_state=2), # vertical flips
    iaa.Crop(percent=(0, 0.1), random_state=3), # random crops
    # Small gaussian blur with random sigma between 0 and 0.5.
    # But we only blur about 50% of all images.
    #iaa.Sometimes(0.5,
          #iaa.GaussianBlur(sigma=(0, 0.5))),

    # Strengthen or weaken the contrast in each image.
    #iaa.contrast.LinearContrast((0.5, 1.25), random_state=4),
    # Add gaussian noise for 50% of the images.
    # For 50% of all images, we sample the noise once per pixel.
    # For the other 50% of all images, we sample the noise per pixel AND
    # channel. This can change the color (not only brightness) of the
    # pixels.
    #iaa.Sometimes(0.5, 
        #iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5), random_state=5),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    #iaa.Multiply((0.8, 1.2), per_channel=0.2, random_state=6),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)}, # random scaling from 80% to 120% of original size
        translate_percent={"x": (0.0, 0.2), "y": (0.0, 0.2)},# translation from 0 translation to 20% of axis size
        rotate=(-360, 360), # rotate the image randomly between -360 and 360 degrees
        shear=(-45, 45),
        cval=255,#set cval to 255 to prevent any black areas occuring 
        mode='constant', random_state=7
    )
], random_state=8) # apply augmenters in random order





In [6]:
'''
test_img_aug = seq_light_img.augment_image(img_arr[0])
test_mask = seq_light_mask.augment_image(masks_arr[0])
f, axarr = plt.subplots(1,2)
axarr[0].imshow(test_img_aug)
axarr[1].imshow(test_mask.reshape(256,256), cmap='gray', vmin=0, vmax=255)
#plt.imshow(test_img_aug)
#plt.imshow(test_mask.reshape((256,256)), cmap='gray', vmin=0, vmax=255)
'''
images_aug_light = seq_light_img.augment_images(img_arr)
masks_aug_light = seq_light_mask.augment_images(masks_arr)


In [7]:
#augment the images
#for light aug
aug_imgs = images_aug_light#images_aug_heavy
aug_masks = masks_aug_light#masks_aug_heavy
#heavy_augments = augment_images_heavy(img_arr)
#print(aug_masks[0].shape)
print(len(aug_imgs))
print(len(aug_masks))


158
158


In [8]:
#check if directory to save the files to exists, if not create one
aug_img_path = './data/duct_test/new_dataset/augmented_images'
aug_mask_path = './data/duct_test/new_dataset/augmented_masks'
if not os.path.isdir(aug_img_path):
    # if the directory doesnt exist, create the directory
    os.mkdir(aug_img_path)
else:
    # delete the contents of the directory so they can be replaced
    imgs_path = aug_img_path + '/*'
    imgs_path = glob.glob(imgs_path)
    for i in imgs_path:
        os.remove(i)

if not os.path.isdir(aug_mask_path):
    os.mkdir(aug_mask_path)
else:
    # delete the contents of the directory so they can be replaced
    mask_path = aug_mask_path + '/*'
    mask_path = glob.glob(mask_path)
    for i in mask_path:
        os.remove(i)
#attain the original filenames
ori_imgs = os.listdir('./data/duct_test/new_dataset/images_unaugmented')
ori_masks = os.listdir('./data/duct_test/new_dataset/masks_unaugmented')

# save the augmented files with the same filenames in the new folder
# saving the augmented originals
for i in range(len(aug_imgs)):
    #use PIL to create an img
    augmented = Image.fromarray(aug_imgs[i], 'RGB')
    #match it to the corresponding name
    img_name = ori_imgs[i]

    #save the image to the directory
    augmented.save(aug_img_path + '/' + img_name)

# saving the augmented masks
print("aug_masks len: " + str(len(aug_masks)))
print("ori_mask_path len: " + str(len(ori_masks)))
aug_masks2 = np.reshape(aug_masks, (len(aug_masks), 256, 256))
print("aug_masks2 len: " + str(len(aug_masks2)))
for i in range(len(aug_masks)):
    augmented = Image.fromarray(aug_masks2[i], 'L')
    mask_name = ori_masks[i]
    augmented.save(aug_mask_path + '/' + mask_name)



aug_masks len: 158
ori_mask_path len: 158
aug_masks2 len: 158
